In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import yaml
import missingno as msno
from icecream import ic
import math

import matplotlib.pyplot as plt
import seaborn as sns   
sns.set(style="whitegrid")

import copy
from tqdm import tqdm

from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

from torch.utils.tensorboard import SummaryWriter
from imblearn.over_sampling import SMOTE

pd.set_option('display.max_columns', None)
%load_ext blackcellmagic
%load_ext autoreload

In [2]:
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [3]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Load data
train_df = pd.read_csv(config["paths"]["ptb_train"], header=None)
test_df = pd.read_csv(config["paths"]["ptb_test"], header=None)

In [5]:
X_train_full = train_df.iloc[:, :-1].to_numpy()
y_train_full = train_df.iloc[:, -1].to_numpy()

X_test = test_df.iloc[:, :-1].to_numpy()
y_test = test_df.iloc[:, -1].to_numpy()

X_train_full = np.c_[X_train_full, np.zeros((X_train_full.shape[0], 3))]
X_test = np.c_[X_test, np.zeros((X_test.shape[0], 3))]

In [6]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=42
)

In [7]:
if config["preprocessing"]["use_smote"]:
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    #ic(X_train.shape, y_train.shape)
    #ic(np.unique(y_train, return_counts=True))

In [8]:
class PTB_Dataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [9]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [10]:
#### ATTENTION
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [11]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.0):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = int(d_model // h)
        self.h = int(h)
        self.linears = clones(nn.Linear(d_model, d_model, dtype=torch.float32), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1) # Same mask applied to all h heads.
            
        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            lin(x).view(-1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears[:3], [query, key, value])
        ]
        
        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = (x.transpose(1, 2).contiguous().view( -1, self.h * self.d_k))
        
        del query
        del key
        del value
        out = self.linears[-1](x)
        return out


In [12]:
## BLOCKING
class LayerNorm(nn.Module):
    "Construct a layernorm module."
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.a_2 = nn.Parameter(torch.ones(features).to(DEVICE))
        self.b_2 = nn.Parameter(torch.zeros(features).to(DEVICE))
        self.eps = eps
    
    def forward(self, x):
        #print(f'Layer Norm input type = {x.dtype}')
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
        #print(f'Layer Norm result type = {result.dtype}')

In [13]:
class SublayerConnection(nn.Module):
    "A residual connection followed by a layer norm."
    def __init__(self, size, dropout):
        super().__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [14]:
class EncoderBlock(nn.Module):
    "Encoder is made up of self-attn and feed forward"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super().__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        ic(x.shape)
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        ic(x.shape)
        x = self.sublayer[1](x, self.feed_forward)
        ic(x.shape)
        return x


In [15]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask=None):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [16]:
class FeedForward(nn.Module):
    "Construct a FeedForward network with one hidden layer"
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff, dtype=torch.float32)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(d_ff, d_model, dtype=torch.float32)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [17]:
class Transformer(nn.Module):
    "Transformer Model"
    def __init__(self, input_size, num_classes, num_heads=8, N=6, d_ff=256, dropout=0.0):
        super().__init__()
        c = copy.deepcopy
        attn = MultiHeadedAttention(num_heads, input_size)
        ff = FeedForward(input_size, d_ff, dropout)
        self.encoder = Encoder(EncoderBlock(input_size, c(attn), c(ff), dropout), N)
        self.fc = nn.Linear(input_size, num_classes, dtype=torch.float32)
    
    def forward(self, x):
        x = self.encoder(x)
        ic(x.shape)
        x = self.fc(x)
        ic(x.shape)
        return x

In [18]:
class EarlyStopping:
    def __init__(self, patience, verbose = 0, mode = 'mim'):
        self.patience = patience
        self.best_metric = float('inf') if mode == 'min' else 0.0
        self.best_epoch = -1
        self.early_stop = False
        self.verbose = verbose
        self.mode = mode

    def __call__(self, val_metric, epoch_index):
        if self.mode == 'max' and val_metric >= self.best_metric or self.mode == 'min' and val_metric <= self.best_metric:
            self.early_stop = False
            self.best_metric = val_metric
            self.best_epoch = epoch_index
        else:
            self.early_stop = (epoch_index - self.best_epoch) >= self.patience
            if self.verbose:
                print(f'EarlyStopping: current epoch {epoch_index + 1} no improvement for metric, best metric = {self.best_metric:0.4f} in epoch = {self.best_epoch + 1}')


In [19]:
train_loader = DataLoader(
    PTB_Dataset(X_train, y_train), batch_size=config["dataloader"]["train_batch_size"], shuffle=True, pin_memory=True
)
val_loader = DataLoader(
    PTB_Dataset(X_val, y_val), batch_size=config["dataloader"]["val_batch_size"], shuffle=False, pin_memory=True
)

test_loader = DataLoader(
    PTB_Dataset(X_test, y_test), batch_size=config["dataloader"]["test_batch_size"], shuffle=False, pin_memory=True
)


In [32]:
def fit_epoch(model, train_loader, criterion, optimizer, epoch_index):
    
    model.train(True) #just in case - decided to switch model to train mode
    
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    batches_total = len(train_loader) #returns number of batches
    y_preds = np.array([])
    y_true = np.array([])
    
    #displaying pretty progress bar for each epoch
    #with tqdm(desc=f"train epoch {epoch_index}:", leave=False, total=batches_total) as pbar_train:
    
    for inputs, labels in train_loader:

            inputs = inputs.to(DEVICE)
            labels = labels.type(torch.LongTensor).to(DEVICE)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            preds = torch.argmax(F.softmax(outputs,1), 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            processed_data += inputs.size(0)
            y_preds = np.concatenate((y_preds, preds.cpu().numpy()))
            y_true = np.concatenate((y_true, labels.cpu().numpy()))
            
            zero_grad_count = 0
            total_grad_count = 0
        #     for param in model.parameters():
        #         if param.grad is not None:
        #                 zero_grad_count += torch.sum(param.grad == 0).item()
        #                 total_grad_count += torch.numel(param.grad)

        #     print(f"Number of zero gradients = {zero_grad_count} / {total_grad_count}")
                
            #pbar_train.update(1)
              
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    train_f1 = f1_score(y_true, y_preds)
    return train_loss, train_acc, train_f1


In [33]:

def eval_epoch(model, val_loader, criterion, epoch_index):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0
    y_preds = np.array([])
    y_true = np.array([])

    #displaying pretty progress bar for each epoch
    #with tqdm(desc=f"eval epoch {epoch_index}:", leave=False, total=len(val_loader)) as pbar_val:
            
    for inputs, labels in val_loader:
            inputs = inputs.to(DEVICE)
            labels = labels.type(torch.LongTensor).to(DEVICE)

            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                preds = torch.argmax(F.softmax(outputs, 1), 1)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            processed_size += inputs.size(0)
            y_preds = np.concatenate((y_preds, preds.cpu().numpy()))
            y_true = np.concatenate((y_true, labels.cpu().numpy()))
            #pbar_val.update(1)
    
    val_loss = running_loss / processed_size
    val_acc = running_corrects.cpu().numpy() / processed_size
    val_f1 = f1_score(y_true, y_preds)
    return val_loss, val_acc, val_f1


In [22]:

def train(train_loader, val_loader, model, epochs, \
          criterion, opt, scheduler = None, early_stopping: EarlyStopping = None):
    
    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f}, val_acc {v_acc:0.4f}, \
    train_f1 {train_f1:0.4f}, val_f1 {val_f1:0.4f} \
    best_val_f1 = {best_val_f1:0.4f}"

    # adding storage for best weights, will be finally used 
    # when training for all epochs is finished
    best_model_wts = model.state_dict()
    best_f1 = 0.0

    with tqdm(desc="epoch", total=epochs) as pbar_outer:

        for epoch in range(epochs):
            
            train_loss, train_acc, train_f1 = fit_epoch(model, train_loader, criterion, opt, epoch)
            
            val_loss, val_acc, val_f1 = eval_epoch(model, val_loader, criterion, epoch)
            history.append((train_loss, train_acc, train_f1, val_loss, val_acc, val_f1))
            
            #remember model weights if current accuracy is better than "best_acc"
            if val_f1 > best_f1:
                best_f1 = val_f1
                best_model_wts = copy.deepcopy(model.state_dict())

            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, \
                                           train_f1 = train_f1, v_acc=val_acc, \
                                           val_f1 = val_f1, best_val_f1 = best_f1))
            
             # if scheduler is defined, do LR step after every epoch
            if scheduler is not None:
                scheduler.step(val_loss)
            
            if early_stopping is not None:
                early_stopping(val_f1, epoch)
                if early_stopping.early_stop:
                    print(f'Early stopping, epoch = {epoch + 1}, best VAL F1 = {best_f1:0.4f}')
                    break
            
    # finally - let's load the best model weights
    model.load_state_dict(best_model_wts)        
            
    return model, history


In [23]:

def predict(model, test_loader):
    
    with torch.no_grad():
        logits = []
        true_labels = []
    
        for inputs, labels in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            true_labels.append(labels.cpu().numpy())
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    true_labels = np.concatenate(true_labels)
    return probs, true_labels

In [24]:

def init_parameters(model):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)


In [25]:
BATCH_SIZE = 32
INPUT_SIZE = X_train.shape[1]
N_CLASSES = len(np.unique(y_train))
DROPOUT = 0.3
LR = 5e-03
N_EPOCHS = 100
HUM_HEADS = 5
NUM_ENCODERS = 5
DIM_FF = 128

model_t = Transformer(INPUT_SIZE, N_CLASSES, HUM_HEADS, NUM_ENCODERS, d_ff = DIM_FF, dropout = DROPOUT)
#init_parameters(model_t)

model_t = model_t.to(DEVICE)

optimizer = optim.Adam(model_t.parameters(), lr=LR)
criterion = torch.nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, threshold=1e-05, verbose = 1)
early_stopping = EarlyStopping(patience = 10, verbose = 1, mode = 'max')

/home/janhsc/miniconda3/envs/ml4hc/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [26]:
model_t, history = train(train_loader, val_loader, model_t, N_EPOCHS,\
                         criterion, optimizer, scheduler, early_stopping)

epoch:   0%|          | 0/100 [00:00<?, ?it/s]ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 2])
/tmp/ipykernel_53046/3330561303.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = torch.argmax(F.softmax(outputs), 1)
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: torch.Size([32, 190])
ic| x.shape: to

KeyboardInterrupt: 

In [ ]:
# testing the model and returning key metrics
probs, y_test = predict(model_t, test_loader)
y_pred = np.argmax(probs, axis=1)
test_f1 = f1_score(y_test, y_pred)
test_acc = np.mean(y_test == y_pred)
test_loss = criterion(torch.tensor(probs), torch.tensor(y_test)).item()

print(f"Test loss: {test_loss}, Test accuracy: {test_acc}, Test F1: {test_f1}")

Test loss: 0.3397256814387403, Test accuracy: 0.9725180350395053, Test F1: 0.9809342230695901


Test loss: 0.34013988103442655, Test accuracy: 0.9728615596015115, Test F1: 0.9810959559703278

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, num_classes, num_heads, num_encoders, dim_feedforward, dropout=0.1):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Linear(input_size, dim_feedforward)
        self.positional_encoding = PositionalEncoding(dim_feedforward, dropout)

        # Custom encoder layer to extract attention
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayerWithAttention(dim_feedforward, num_heads, dropout)
            for _ in range(num_encoders)
        ])
        
        self.output_layer = nn.Linear(dim_feedforward, num_classes)
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        attentions = []
        
        for layer in self.encoder_layers:
            x, attn = layer(x)
            attentions.append(attn)
        
        x = x.mean(dim=1)
        x = self.output_layer(x)
        return x#, attentions

class TransformerEncoderLayerWithAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.linear1 = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_model, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src):
        src2, attn = self.self_attn(src, src, src, need_weights=True)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.dropout(self.linear1(F.relu(self.linear2(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src, attn


In [ ]:
def create_padding_mask(seq, pad_token=0):
    # Assuming `seq` is of shape [batch_size, sequence_length]
    # and padded positions are denoted by `pad_token` (e.g., 0)
    mask = (seq != pad_token)  # Shape: [batch_size, 1, 1, sequence_length]
    # `True` for non-pad tokens and `False` for pad tokens
    return mask

In [60]:
class TransformerModel(nn.Module):
    def __init__(
        self, input_size, model_size, num_classes, num_heads=8, num_layers=6, d_ff=256, dropout=0.1
    ):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_size, input_size * model_size)
        #self.pos_encoder = PositionalEncoding(model_size, dropout)
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=model_size, nhead=num_heads, dim_feedforward = d_ff, 
            dropout=dropout, activation='gelu', layer_norm_eps=1e-6
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layers, num_layers=num_layers,
            norm = nn.LayerNorm(model_size)
        )
        self.output_layer = nn.Linear(2 * model_size, num_classes)
        self.input_size = input_size

    def forward(self, x):
        src = self.embedding(x) * math.sqrt(self.input_size)
        src = src.reshape(x.shape[0], x.shape[1], -1)
        #ic(src.shape)
        #src = self.pos_encoder(src)
        #ic(src.shape)
        output = self.transformer_encoder(src)
        #ic(output.shape)
        #ic(output.mean(dim=1).shape)
        # max pooling:
        avg_pooled = output.mean(dim=1)
        max_pooled = output.max(dim=1).values
        output = torch.cat([avg_pooled, max_pooled], dim=1)
        output = self.output_layer(output)
        #ic(output.shape)
        return output

In [54]:
def init_parameters(model):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

In [63]:
BATCH_SIZE = 32
INPUT_SIZE = X_train.shape[1]
N_CLASSES = len(np.unique(y_train))
DROPOUT = 0.2
LR = 1e-3
N_EPOCHS = 100
NUM_HEADS = 5
NUM_ENCODERS = 5
DIM_FF = 128
MODEL_SIZE = 10

model_t = TransformerModel(
    num_classes=N_CLASSES,
    input_size=INPUT_SIZE,
    model_size=MODEL_SIZE,
    num_heads=NUM_HEADS,
    num_layers=NUM_ENCODERS,
    d_ff=DIM_FF,
    dropout=DROPOUT
)
model_t = model_t.to(DEVICE)
init_parameters(model_t)

optimizer = optim.AdamW(model_t.parameters(), lr=LR, weight_decay = config["adamw"]["weight_decay"])
criterion = torch.nn.CrossEntropyLoss()
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, threshold=1e-05, verbose = 1)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
early_stopping = EarlyStopping(patience = 10, verbose = 1, mode = 'max')

/home/janhsc/miniconda3/envs/ml4hc/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [64]:
model_t, history = train(train_loader, val_loader, model_t, N_EPOCHS,\
                         criterion, optimizer, scheduler=None, early_stopping=early_stopping )


Epoch 001 train_loss: 0.5561     val_loss 0.4443 train_acc 0.7201, val_acc 0.7935,     train_f1 0.8299, val_f1 0.8589     best_val_f1 = 0.8589



Epoch 002 train_loss: 0.4752     val_loss 0.3934 train_acc 0.7529, val_acc 0.8042,     train_f1 0.8380, val_f1 0.8611     best_val_f1 = 0.8611



Epoch 003 train_loss: 0.4267     val_loss 0.4066 train_acc 0.7908, val_acc 0.7978,     train_f1 0.8570, val_f1 0.8556     best_val_f1 = 0.8611


epoch:   3%|▎         | 3/100 [00:06<03:26,  2.13s/it]

EarlyStopping: current epoch 3 no improvement for metric, best metric = 0.8611 in epoch = 2


...


Epoch 065 train_loss: 0.0920     val_loss 0.1482 train_acc 0.9661, val_acc 0.9493,     train_f1 0.9765, val_f1 0.9650     best_val_f1 = 0.9665


epoch:  65%|██████▌   | 65/100 [02:18<01:15,  2.14s/it]

EarlyStopping: current epoch 65 no improvement for metric, best metric = 0.9665 in epoch = 63


epoch:  65%|██████▌   | 65/100 [02:20<01:15,  2.16s/it]


KeyboardInterrupt: 